In [1]:
#Import modules
import numpy as np
from surprise import SVD
from surprise.model_selection import cross_validate
from surprise import Dataset
import pandas as pd
from surprise import Reader
from surprise.model_selection.split import PredefinedKFold
from surprise import accuracy
from surprise.prediction_algorithms.knns import KNNBasic

In [2]:
#Load Data
folds_files = [('train.txt','test.txt')]
reader = Reader(line_format='user item rating', sep='\t', skip_lines=None, rating_scale=(1, 5))
data = Dataset.load_from_folds(folds_files,reader=reader)
pkf = PredefinedKFold()
trainset, testset = next(pkf.split(data))

In [7]:
#Set params
k=20
epochs = 30
lambdas = 0.1
eta = 0.03
algo = SVD(n_factors=k,n_epochs=epochs,reg_all=0.1,lr_all=eta)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions,verbose=True)

#Set params
k=20
epochs = 30
lambdas = 0.1
algo = SVD(n_factors=k,n_epochs=epochs,biased=False,reg_all=0.1,lr_all=eta)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions,verbose=True)

algo.fit()


RMSE: 0.9143
RMSE: 0.9458


0.9458012314497669

In [5]:
#Load Data
k=20
epochs = 30
lambdas = 0.1
eta = 0.03
data = Dataset.load_from_file('data.txt',reader=reader)
trainset = data.build_full_trainset()
algo = SVD(n_factors=k,n_epochs=epochs,biased=False,reg_all=0.1,lr_all=eta)
algo.fit(trainset)
V_surprise = algo.qi
U_surprise = algo.pu